<table style="border: none; border-collapse: collapse;" border="0" cellspacing="0" cellpadding="0" width="100%" align="center">
<tr style="border: none;">
<td  style="border: none;" width="20%">Brought to you by <h3>the dynamic trio</h3></td>
<td style="border: none;"><a href="https://github.com/fredokun/cl-jupyter"><img src="http://jupyter.org/assets/nav_logo.svg" alt="Project Jupyter" height="128"></a></td>
<td style="border: none;"><a href="http://clhs.lisp.se/"><img src="http://www.lisperati.com/lisplogo_fancy_128.png" alt="Public Domain Lisp Logo Set By Conrad Barski, M.D." height="128"></a></td><td style="border: none;"><a href="https://www.hdfgroup.org/"><img src="https://www.hdfgroup.org/images/hdf_logo.jpg" alt="The HDF Group" height="128"></a></td></tr>
</table>

# Reading the Root Group's Object Header

The primitives defined in the [HDF5 File Format Specification](https://www.hdfgroup.org/HDF5/doc/H5.format.html) are organized into three levels.

> Level 0 contains basic information for identifying and defining information about the file. Level 1 information contains the information about the pieces of a file shared by many objects in the file (such as a B-trees and heaps). Level 2 is the rest of the file and contains all of the data objects, with each object partitioned into header information, also known as *metadata*, and data.

In this installment, we skip level 1, and get a preview of a level 2 primitve. The superblock (level 0) contains the address of the root group's [*object header*](https://www.hdfgroup.org/HDF5/doc/H5.format.html#DataObject) (OHDR). What appears from the perspective of the HDF5 library as a collection of links, is represented in the file as a conglomerate of [*object header messages*](https://www.hdfgroup.org/HDF5/doc/H5.format.html#ObjectHeaderMessages) and supporting level 1 primitives. There are about two dozen types of messages, which make up the "vocabulary" of an "object description language." Not all message combinations are grammatically correct, for example, an object header in which a [Group Info Message](https://www.hdfgroup.org/HDF5/doc/H5.format.html#GroupInfoMessage) were followed by a [Dataspace Message](https://www.hdfgroup.org/HDF5/doc/H5.format.html#DataspaceMessage) would be as nonsensical as *Quadruplicity drinks procrastination.* (B. Russell)

We go through the usual motions to create a simple test file:

In [1]:
(defparameter *file-name* "root-ohdr.h5")
(let ((fcpl (h5pcreate +H5P-FILE-CREATE+)))
     (h5pset-userblock fcpl (cffi:convert-to-foreign 524288 'hsize-t))
     (h5fclose (h5fcreate *file-name* +H5F-ACC-TRUNC+ fcpl +H5P-DEFAULT+))
     (h5pclose fcpl))
(defparameter *input-stream*
    (open *file-name* :direction :input :element-type '(unsigned-byte 8)))
(defparameter *file* (make-file *file-name*))
(add-superblock *input-stream* *file*)
(values)

NIL

The address of the root group's object header is stored in the file superblock.

In [2]:
(get-root-ohdr-address *file*)

524384

Except for the role of being the *origin* or base node of an HDF5 multi-graph, there is nothing special about the root group. It's an HDF5 group. In the `alcove` package, we have defined a simple function, `READ-OHDR`, for reading HDF5 object headers.

In [3]:
(documentation 'read-ohdr 'function)

"--------------------------------------------------------------[function-doc]
READ-OHDR
Args: (INPUT-STREAM ADDRESS FILE)
Read an object header (OHDR) from INPUT-STREAM at ADDRESS.
Return an association list.
------------------------------------------------------------------------------"

In [4]:
(loop for x in (read-ohdr *input-stream* (get-root-ohdr-address *file*) *file*)
   do (format t "~a~%" x))

(OHDR-VERSION . 1)
(MESSAGE-COUNT . 1)
(REFERENCE-COUNT . 1)
(HEADER-SIZE . 24)
(MESSAGES
  ((MSG-TYPE . 17) (MSG-DATA-SIZE . 16) (MSG-FLAGS . 0)
   (MSG-DATA (NAME . Symbol Table) (V1-B-TREE-ADDRESS . 136)
    (LOCAL-HEAP-ADDRESS . 680))))


NIL

As most other primitives in the HDF5 file format, the object header is versioned. There's only one message stored in this particular object header, and the HDF5 object has a reference count of 1. The message is a [Symbol Table Message](https://www.hdfgroup.org/HDF5/doc/H5.format.html#SymbolTableMessage), which tells us that the corresponding HDF5 object is indeed an (old-style) HDF5 group. Information about the links in the group is stored in two places: A B-tree manages references to link names and descriptions of link destinations, and the actual names and descriptions are stored on a local heap. We'll dig into reading those items in the next installment.

In [5]:
(close *input-stream*)

T

### The Root Group's Object Header Version 2

Let's take a look at the object header of a "newer style" HDF5 group.

In [6]:
(setq *file-name* "root-ohdr.v2.h5")
(let ((fapl (h5pcreate +H5P-FILE-ACCESS+))
      (fcpl (h5pcreate +H5P-FILE-CREATE+)))
  (h5pset-libver-bounds fapl :H5F-LIBVER-LATEST :H5F-LIBVER-LATEST)
  (h5pset-userblock fcpl (cffi:convert-to-foreign 1024 'hsize-t))
  (h5fclose (h5fcreate *file-name* +H5F-ACC-TRUNC+ fcpl fapl))
  (h5pclose fcpl)
  (h5pclose fapl))
(setq *input-stream*
      (open *file-name* :direction :input :element-type '(unsigned-byte 8)))
(setq *file* (make-file *file-name*))
(add-superblock *input-stream* *file*)
(values)

NIL

In [7]:
(loop for x in (read-ohdr *input-stream* (get-root-ohdr-address *file*) *file*)
   do (format t "~a~%" x))

(SIGNATURE . OHDR)
(OHDR-VERSION . 2)
(FLAGS . 32)
(ACCESS-TIME . 1457828958)
(MODIFICATION-TIME . 1457828958)
(CHANGE-TIME . 1457828958)
(BIRTH-TIME . 1457828958)
(SIZE-OF-CHUNK0 . 120)
(MESSAGES
  ((MSG-TYPE . 0) (MSG-DATA-SIZE . 88) (MSG-FLAGS . 0)
   (MSG-DATA (NAME . NIL)
    (DATA
     . #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
         0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
         0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0))))
  ((MSG-TYPE . 10) (MSG-DATA-SIZE . 2) (MSG-FLAGS . 1)
   (MSG-DATA (NAME . Group Info) (VERSION . 0) (FLAGS . 0)))
  ((MSG-TYPE . 2) (MSG-DATA-SIZE . 18) (MSG-FLAGS . 0)
   (MSG-DATA (NAME . Link Info) (VERSION . 0) (FLAGS . 0)
    (FRACTAL-HEAP-ADDRESS . 18446744073709551615)
    (NAME-INDEX-V2-B-TREE-ADDRESS . 18446744073709551615))))
(CHECKSUM . #(215 128 248 180))


NIL

There's a lot more going on in a [version 2 object header](https://www.hdfgroup.org/HDF5/doc/H5.format.html#V2ObjectHeaderPrefix)! This version saw the introduction of the following:

- a signature
- time stamps
- the support for arbitrarily large attributes
- facilities for the tracking and indexing of attributes by creation time
- an object header checksum

In this particular case, the time stamps are stored, as is confirmed by a flag value of 32 (bit 5 is set). There are three messages stored in this object header.

1. The [NIL Message](https://www.hdfgroup.org/HDF5/doc/H5.format.html#NILMessage) can be ignored (and has nothing to do with the Common Lisp constant `NIL`). 
2. The [Group Info Message](https://www.hdfgroup.org/HDF5/doc/H5.format.html#GroupInfoMessage) confirms that we are dealing with a "new style" HDF5 group.
3. The [Link Info Message](https://www.hdfgroup.org/HDF5/doc/H5.format.html#LinkInfoMessage) stores references to variable size information about the group. In this particular example, there are no links in the group. We'll read that variable information in a future installment.

In [8]:
(close *input-stream*)

T